# 智能手机的人类活动识别 (Tensorflow) 

人类活动识别数据库是根据30名参加日常生活活动（ADL）的研究参与者的录音资料建立的，同时携带了带有嵌入式惯性传感器的腰挂式智能手机。目标是将活动分类为所执行的六项活动之一。

### 实验描述

这项实验是在一组年龄在19-48岁之间的30名志愿者中进行的。每个人在腰上佩戴智能手机（三星Galaxy S II）进行了六项活动（步行，步行，用餐，步行，坐，站立，放置）。使用嵌入式加速度计和陀螺仪，我们以50Hz的恒定速率捕获3轴线性加速度和3轴角速度。这些实验已经过视频记录以手动标记数据。所获得的数据集已被随机分成两组，其中70％的志愿者被选中用于生成训练数据和30％的测试数据。

传感器信号（加速度计和陀螺仪）通过应用噪声滤波器进行预处理，然后在2.56秒和50％重叠（128个读数/窗口）的固定宽度滑动窗口中进行采样。具有重力和身体运动分量的传感器加速度信号通过巴特沃斯低通滤波器分离成体加速度和重力。假定重力仅具有低频分量，因此使用具有0.3Hz截止频率的滤波器。从每个窗口，通过计算来自时间和频率域的变量获得特征向量。

## Import Libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap

# 使用plotly  交互式图表
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import pdb
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train.csv") # (7352, 563)
test = pd.read_csv("../input/test.csv") # (2947, 563)

In [ ]:
train.head()

## Preprocessing

Random shuffle the dataset, because the raw dataset have the data from same subject & activities grouped together. 

In [ ]:
train = train.sample(frac=1)
test = test.sample(frac=1)

Drop the last two columns:'subject' and 'Activity', the 'Activity' is our target label

In [ ]:
Y_train = train.Activity
train = train.drop(['Activity','subject'], axis=1) #(7352, 561)

Y_test = test.Activity
test = test.drop(['Activity','subject'], axis=1) #(2947, 561)

## Labels Distributed

In [ ]:
# Plotting data
label_counts = Y_train.value_counts()

# Get colors
n = label_counts.shape[0]
colormap = get_cmap('summer')
colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

# Create plot
data = go.Bar(x = label_counts.index,
              y = label_counts,
              marker = dict(color = colors))

layout = go.Layout(title = 'Smartphone Activity Label Distribution',
                   xaxis = dict(title = 'Activity'),
                   yaxis = dict(title = 'Count'))

fig = go.Figure(data=[data], layout=layout)
iplot(fig)

## KMeans cluster

In [ ]:
# initit the centroid
def generate_initial_centroid(samples, num_cluster):
  '''
  This is the first step of K-means. 
  samples: tensor (num_samples, num_features)
  num_cluster: K, number of clusters.
  
  '''
  num_samples = tf.shape(samples)[0]
  random_indices = tf.random_shuffle(tf.range(0, num_samples))
  centroid_indices = tf.slice(random_indices, [0], [num_cluster])
  init_centroid = tf.gather(samples, centroid_indices)
  
  return init_centroid
  

*Test the initial cluster generating function*

In [ ]:
init_test = train.iloc[:1500].values

init_placeholder = tf.placeholder(tf.float32, [1500, 561])
init_res = generate_initial_centroid(init_placeholder, 6)
with tf.Session() as sess:
  init_centroid = sess.run(init_res, feed_dict={init_placeholder:init_test})
  
print("The expected shape is (6, 561) and the generated shape is {0}".format(init_centroid.shape))

### Assgin each sample to its nearest centroid

In [ ]:
# Check broadcasting rules. 

a = np.array([[[1,2,3],[4,5,6]]]) # (1,2,3) -- (1, num_samples, num_features)
b = np.array([[[1,1,1]],[[4,4,4]]]) # (2,1,3) -- (num_centroids, 1, num_features)
print("The result of a - b is \n{0}".format(a - b))
print("The shape of a - b is {0}".format((a-b).shape))

In [ ]:
def assign_to_nearest(samples, centroids):
  """
  This function assign each sample to its nearest centroid. 
  samples: tensor, (num_samples, num_features)
  centroids: tensor, (num_centroids, num_features)
  """
  expend_samples = tf.expand_dims(samples, 0) # samples become (1, num_samples, num_features)
  expend_centroid = tf.expand_dims(centroids, 1) # centroid become (num_centroid, 1, num_features)
  
  ## each entry represents how far a sample to a centroid. 
  distances = tf.reduce_sum(tf.square(tf.subtract(expend_samples, expend_centroid)), 2) # distance: (num_centroid, num_samples)
  
  ## which centorid each sample is assigned to. 
  nearest_index = tf.argmin(distances, 0) # nearest_index:(num_samples)
  
  return nearest_index

In [ ]:
assign_samples = tf.constant(np.array([[1,2,3],[4,5,6]]))
assign_centroid = tf.constant(np.array([[1,1,1],[4,4,4]]))
with tf.Session() as sess:
  assign_nearest_index = assign_to_nearest(assign_samples, assign_centroid)
  assign_res = sess.run(assign_nearest_index)

print("The expected output is (0,1), and the actual output is {0}".format(assign_res))
print("The first sample (1,2,3) should be assigned to centroid (1,1,1)")
print("The second sample (4,5,5) should be assigned to centroid (4,4,4)")

### Get new centroids by averaging out the assignment.

In [ ]:
def update_centroid(samples, nearest_index, num_clusters):
  """
  samples: tensor, (num_samples, num_features)
  nearest_index: tensor, (num_samples)
  num_clusters: int
  """
  
  nearest_index = tf.to_int32(nearest_index)
  partitions = tf.dynamic_partition(samples, nearest_index, num_clusters)
  new_centroids = tf.concat([tf.reduce_mean(partition, 0, keep_dims=True) for partition in partitions], axis=0)
  
  return new_centroids, nearest_index

*Test the update function*

In [ ]:
# Test the function: update_centroid.
with tf.Session() as sess:
  new_cent, _ = update_centroid(assign_samples, assign_res, 2)
  update_res = sess.run(new_cent)
  
print("The expected new centroids are (1,2,3), (4,5,6)")
print("The actual new centroids are \n{0}".format(update_res))

### Run the K-means algorithm

In [ ]:
k_means_placeholder = tf.placeholder(tf.float32, shape=(7352, 561))
updated_centroids = tf.placeholder(tf.float32, shape=(6, 561))

init_centroids = generate_initial_centroid(k_means_placeholder, num_cluster=6)


nearest_index = assign_to_nearest(k_means_placeholder,updated_centroids)
updated_centroid = update_centroid(k_means_placeholder, nearest_index, 6)

with tf.Session() as sess:
  centroids = sess.run(init_centroids, feed_dict={k_means_placeholder:train})
  for i in range(0, 300):
    
    centroids,nearest_index = sess.run(updated_centroid, feed_dict={k_means_placeholder:train,
                                                         updated_centroids:centroids})
    

In [ ]:
print(nearest_index)
print(Y_train.values)

tsne_data = train.copy()
scl = StandardScaler()
tsne_data = scl.fit_transform(tsne_data)

tsne = TSNE(random_state=3)
tsne_transformed = tsne.fit_transform(tsne_data)

# Get colors
n = label_counts.shape[0]
colormap = get_cmap('viridis')
colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

plt.figure(figsize=(20,15))

for i, group in enumerate(label_counts.index):
    # Mask to separate sets
    mask = (nearest_index==i)
    plt.scatter(x=tsne_transformed[mask][:,0], y=tsne_transformed[mask][:,1], c=colors[i], alpha=0.5, label=group)

plt.title('KMeans Cluster Visualisation')
plt.show()
    
pd.crosstab(nearest_index, Y_train)

## Plot Activities

In [ ]:
### Plot Activities
# Get colors
n = label_counts.shape[0]
colormap = get_cmap('viridis')
colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

plt.figure(figsize=(20,15))

for i, group in enumerate(label_counts.index):
    # Mask to separate sets
    mask = (Y_train==group).values
    plt.scatter(x=tsne_transformed[mask][:,0], y=tsne_transformed[mask][:,1], c=colors[i], alpha=0.5, label=group)
plt.title('TSNE Activity Visualisation')
plt.show()

Take out message:

1) We can see that basically walking activities are separated out from the passive activities. Cluster 0 and Cluster 3 are filled with walking activities, while the rest clusters represent the passive activities. 

2) Cluster 0 has more 'walking' and 'walking upstairs' in it, while majority of samples of cluster 3 are 'walking downstairs'. That means 'walking downstairs' are easier to separate out from the other two, while the 'walking' and 'walking upstair' are very easy to get mixed.

3) Cluster 2 and 5 are filled with 'laying', and cluster 1 and 4 are filled with sitting and standing. Laying is clearly easier to distinguished from the other two. And it seems that k-means algorithm has difficulites identifying sitting and standing. 

## Neural Network Model

One-hot encoded our target label

In [ ]:
Y_train = pd.get_dummies(Y_train)
Y_test = pd.get_dummies(Y_test)

train = train.as_matrix()
test = test.as_matrix()

Y_train = Y_train.as_matrix()
Y_test = Y_test.as_matrix()

In [ ]:
FEATURE_DIM = 561
LEARNING_RATE = 0.001
LABEL_DIM = 6
BATCH_SIZE = 64
NUM_EPOCH = 100

### Network Object

In [ ]:
class Neural_Network():
  
  def __init__(self, feature_dim = FEATURE_DIM, label_dim = LABEL_DIM):
    self.feature_dim = feature_dim
    self.label_dim = label_dim
    
    
  def build_network(self, learning_rate=LEARNING_RATE):
    
    self.train_X = tf.placeholder(tf.float32, [None, self.feature_dim])
    self.train_Y = tf.placeholder(tf.float32, [None, self.label_dim])
    
    self.layer_1 = self.dense_layer(self.train_X, self.feature_dim, 
                                    1024, activation=tf.nn.relu, name='layer_1')
    self.layer_2 = self.dense_layer(self.layer_1, 1024, 512, 
                                   activation=tf.nn.relu, name='layer_2')
    self.layer_3 = self.dense_layer(self.layer_2, 512, 64, 
                                   activation=tf.nn.relu, name='layer_3')
    self.output = self.dense_layer(self.layer_3, 64, 6, name='output')
    
    self.loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=self.output, labels = self.train_Y))
    
    self.optimizer = tf.train.AdamOptimizer(learning_rate)
    
    self.train_step = self.optimizer.minimize(self.loss)
    
    self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.output,1), 
                                                    tf.argmax(self.train_Y, 1)),'float'))
    
  def dense_layer(self, inputs, input_size, output_size, name, activation=None):
    
    W = tf.get_variable(name=name+'_w',shape=(input_size, output_size), 
                        initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name=name+'_b', shape=(output_size))
    out = tf.matmul(inputs, W) + b
    
    if activation:
      return activation(out)
    else:
      return out
    

### Data Object

In [ ]:
class Data():
  
  def __init__(self, train_X, train_Y, batch_size=BATCH_SIZE):
    
    self.train_X = train_X
    self.train_Y = train_Y
    self.batch_size = batch_size
    self.num_batch = self.train_X.shape[0]//batch_size
    
  def generate_batch(self):
    
    for i in range(self.num_batch):
      
      x = self.train_X[(i*self.batch_size):(i+1)*self.batch_size, :]
      y = self.train_Y[(i*self.batch_size):(i+1)*self.batch_size]
      
      yield x, y 
    

### Training Object

In [ ]:
class Learn():
  
  def __init__(self, train_X, train_Y, test_X, test_Y, 
               batch_size=BATCH_SIZE, epoch = NUM_EPOCH):
    
    self.batch_size = batch_size
    self.epoch = epoch
    
    self.network = Neural_Network()
    self.network.build_network(learning_rate=0.001)
    self.data = Data(train_X, train_Y, self.batch_size)
    self.test_X = test_X
    self.test_Y = test_Y
  
  def run_training(self):
    init = tf.initialize_all_variables()
    
    with tf.Session() as sess:
      
      sess.run(init)
      
      training_loss = []
      counter, tmp_loss = 0, 0
      
      for i in range(self.epoch):
        
        for x, y in self.data.generate_batch():
          
          feed_dict = {self.network.train_X:x, self.network.train_Y:y}
        
          _, loss = sess.run([self.network.train_step, self.network.loss], 
                             feed_dict=feed_dict)
          
          if counter % 100 == 0 and counter!=0:
            training_loss.append(tmp_loss/100)
            tmp_loss = 0
          else:
            tmp_loss += loss
            
          counter += 1
          
        print("Epoch {0}, loss is {1}".format(i, loss))
        
        
        
      self.training_loss = training_loss
      acc = sess.run([self.network.accuracy], feed_dict={self.network.train_X:self.test_X,
                                                        self.network.train_Y:self.test_Y})
      print("The testing accuracy is {0}".format(acc))
      
  def plot_training_loss(self):
    plt.plot(self.training_loss)

In [ ]:
tf.reset_default_graph()

learner = Learn(train, Y_train, test, Y_test, epoch=NUM_EPOCH)
learner.run_training()
learner.plot_training_loss()

可见，随着Epoch增加，loss值快速降低并逐渐趋向稳定，训练效果良好，最后在测试集的准确值为95%左右。